<a href="https://www.kaggle.com/code/emdogan/915-deberta3base-training-tercume?scriptVersionId=166933418" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

## 🛑 Wait a second - after this you should also look at the inference notebook
- My inference notebook (containing equally many emojis) is here:
- https://www.kaggle.com/code/valentinwerner/893-deberta3base-inference

## 🏟️ Credits (because this baseline did mostly already exist when I joiend)

- @Nicholas Broad published the transformer baseline which performs only marginally worse: https://www.kaggle.com/code/nbroad/transformer-ner-baseline-lb-0-854
- @Joseph Josia published the training notebook which I basically copy pasted (which is based itself on nbroad, but yeah): https://www.kaggle.com/code/takanashihumbert/piidd-deberta-model-starter-training



## 💡 What I added
- Downsampling negative samples (samples without labels, but they possible still work as examples where names should not be tagged as name)
- Adding @moths external data: https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/469493
- Adding PJMathematicianss external data: https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/470921
- However, I used my cleaned version instead (the punctuation is flawed in the original data set at the time of this trainign): https://www.kaggle.com/code/valentinwerner/fix-punctuation-tokenization-external-dataset

Doing this brought the LB score to .888 - Trained in Kaggle Notebook, no tricks or secrets.

- I added emojis because that seems to be the kaggle upvote meta

## 📝 Config & Imports
- 1024 max length has been working well for me. As some samples are longer, you may want to go as high as you can 

In [ ]:
TRAINING_MODEL_PATH = "microsoft/deberta-v3-base"
TRAINING_MAX_LENGTH = 1024
OUTPUT_DIR = "output"
"""
-------------------------------------
- Model için ayarlamalar ve import gerçekleştirilmiş.

+ TRAINING_MODEL_PATH: eğitimde kullanılacak modelin adı bu değişkene atanmış, DeBERTa modeli kullanılmış.
+ TRAINING_MAX_LENGTH: eğitimde kullanılacak maksimum giriş uzunluğunu belirlenmiş.
+ OUTPUT_DIR: eğitim sırasında oluşturulan çıktı dosyalarının kaydedileceği dizin belirtilmiş, çıktıların "output" adlı bir dizine 
  kaydedileceği belirlenmiş.
-------------------------------------
"""

In [ ]:
!pip install seqeval evaluate -q

In [ ]:
import json
import argparse
from itertools import chain
from functools import partial

import torch
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
import evaluate
from datasets import Dataset, features
import numpy as np

## 🗺️ Data Selection and Label Mapping
- As mentioned before, I additionaly use the moth dataset

In [ ]:
data = json.load(open("/kaggle/input/pii-detection-removal-from-educational-data/train.json"))

# downsampling of negative examples
p=[] # positive samples (contain relevant labels)
n=[] # negative samples (presumably contain entities that are possibly wrongly classified as entity)
for d in data:
    if any(np.array(d["labels"]) != "O"): p.append(d)
    else: n.append(d)
        
# orjinal örnek sayısı
print("original datapoints: ", len(data))

# ek veri setinin örnek sayısı
external = json.load(open("/kaggle/input/fix-punctuation-tokenization-external-dataset/pii_dataset_fixed.json"))
print("external datapoints: ", len(external))

# ek veri setinin örnek sayısı
moredata = json.load(open("/kaggle/input/fix-punctuation-tokenization-external-dataset/moredata_dataset_fixed.json"))
print("moredata datapoints: ", len(moredata))

# tüm veri setinin örnek sayısı
data = moredata+external+p+n[:len(n)//3]
print("combined: ", len(data))
"""
--------------------------
- ekstra datasetler import edilmiş ve örnek sayıları ekrana bastırılmış

+ "data" değişkenine yarışmanın kendi train seti import edilmiş
+ Pozitif ve negatif değerler "p" ve "n" listelerine atanıcak
  + data değişkeni üzerinde döngü başlatılmış
  + "O" etiketine sahip olmayan etiketler pozitif yani "p" listedine eklenmiş
  + "O" etiketleri ise negatif yani "n" değişkenine atanmış
  + döngü bittikten sonra orjinal data sayısını yani örnek sayısını ekrana printlemiş
+ dışardan dataset elde edilip "external" değişkenine import edilmiş
  + ekrana bu ds'in kaç örnek içerdiği yazdırılmış
+ "moredata" değişkenine dışardan başka ds yüklenmiş
  + ekrana kaç örnek içerdiği yazdırılmış
+ "data" değişkenine dışardan import edilen ds'ler dahil olmak üzere "p" ve "n" listeleri de dahil edilerek birleştirme yapılmış
  + combined adı altında tüm verilerin örnek sayıları ekrana yazdırılmış
--------------------------
"""

In [ ]:
####
all_labels = sorted(list(set(chain(*[x["labels"] for x in data]))))
label2id = {l: i for i,l in enumerate(all_labels)}
id2label = {v:k for k,v in label2id.items()}

target = [
    'B-EMAIL', 'B-ID_NUM', 'B-NAME_STUDENT', 'B-PHONE_NUM', 
    'B-STREET_ADDRESS', 'B-URL_PERSONAL', 'B-USERNAME', 'I-ID_NUM', 
    'I-NAME_STUDENT', 'I-PHONE_NUM', 'I-STREET_ADDRESS', 'I-URL_PERSONAL'
]

# print(id2label)

"""
----------------------------
- ds'lerimiz ile önce tüm label'Ların listesi oluşturulmuş, sonradında her bir etiketin bir indeksle eşleştirilmiş hali olan label2id ve 
  indekslerin etiketlere eşleştirilmiş hali olan id2label sözlüklerini oluşturulmuş.

+ tüm örnekleri içeren "data" değişkenini döngüye alarak chain fonksiyonu ile labels etiketleri tek liste haline getirilmiş, "all_labels" 
  listesine atanmış
  + set fonksiyonu ile listedeki labels'lar uniq hale getirilmiş yani her bir etiket listeye 1 defa eklenmiş
  + list fonksiyonu ile oluşturulan set bir liste haline getirilmiş
  + sorted fonksiyonu ile bu liste alfabetik olarak sıralanmış
+ "all_labels" listesinden enumerate ile "label2id" oluşturulmuş
+ "label2id" kullanılarak tam tersi yani id2label oluşturulmuş
+ target adında bir liste oluşturulmuş ve bu listede aradığımız etiketler sıralanmış
----------------------------
"""

## ♟️ Data Tokenization
- This tokenizer is actually special, comparing to usual NLP challenges

In [ ]:
def tokenize(example, tokenizer, label2id, max_length):

    # rebuild text from tokens
    text = []
    labels = []

    for t, l, ws in zip(
        example["tokens"], example["provided_labels"], example["trailing_whitespace"]
    ):
        text.append(t)
        labels.extend([l] * len(t))

        if ws:
            text.append(" ")
            labels.append("O")

    """
    -------------------------------------------------------
        + "text" ve "labels" listeleri oluşturulmuş
        + "example" sözlüğünden "text", "labels" ve "trailing_whitespace" değişkenleri ile döngü oluşturulmuş
        + her bir token "text" listesine eklenmiş
        + tokenin uzunluğu kadar aynı etiket birden çok kez labels'a eklenmiş çünkü her bir karakter için aynı etiket geçerli.
        + eğer tokenin ardından boşluk karakteri varsa, boşluk karakteri text listesine eklenmiş ve labels listesine de "o" etiketi 
          eklenmiş
    -------------------------------------------------------
    """

    # actual tokenization
    tokenized = tokenizer("".join(text), return_offsets_mapping=True, max_length=max_length)

    labels = np.array(labels)

    text = "".join(text)
    token_labels = []
    """
    + tokenizer çağırılarak metin tokenize edilmiş ve "tokenized" değişkenine atanmış
    + "".join(text) ile metindeki tokenler birleştirilip tek dize hale getirilmiş
    + return_offsets_mapping=True tokenin başlangıç ve bitiş konumlarını takip eden offsetlerin return edilmesi sağlanmış
    + uzunluk olarak daha önce oluşturduğumuz "max_length" değişkeni kullanılmış
    + "labels" değişkenine etiketler numpy arry olarak atanmış
    + text değişkenine tüm tokenler birleştirilerek tekrardan atama yapılmış
    + "token_labels" adında boş liste oluşturulmuş
    """
    for start_idx, end_idx in tokenized.offset_mapping:
        # CLS token
        if start_idx == 0 and end_idx == 0:
            token_labels.append(label2id["O"])
            continue

        # case when token starts with whitespace
        if text[start_idx].isspace():
            start_idx += 1

        token_labels.append(label2id[labels[start_idx]])

    length = len(tokenized.input_ids)

    return {**tokenized, "labels": token_labels, "length": length}

"""
-----------------------------
- tokenized fonksiyonu oluşturulmuş metin, tokenizer, label2id ve uzunluk bilgilerini alarak metni tokenize etmemizi sağlamış, 
  return olarak tokenized metni, etiketleri ve uzunluklarını sözlük biçiminde döndürmüş.
  

+ tokenin offset maplerini kullanarak etiketlerini belirlememizi sağlayan bir döngü başlatılmış
  + "start_idx" ve "end_idx" tokenin orjinal metindeki başlangıç ve bitiş konumlarını ifade ediyor
  + if şartında tokenin CLS belirteçi olup olmadığını kontrol edilmiş
  + eğer token CSL belirteçi ise tokenin etiketi other "O" olarak belirlenir
+ eğer token boşluk ile başlamışsa başlangıç indeksine +1 eklenmiş
+ indeksteki tokenin etiketi label2id sözlüğünden alınarak token_labels listesine eklenmiş
+ tokenized sözlüğü içindeki input_ids öğesinin uzunluğu hesaplanmış, tokenleştirilmiş metindeki toplam token sayısı "length" değişkenine 
  atanmış.
+ return olarak sözlük döndürülmüş
  + tokenized sözlüğündeki tüm ögeler kapsanmış
  + labels yani token etiketleri döndürülmüş
  + lenght yani metindeki toplam token sayısı döndürülmüş
-----------------------------
"""

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(TRAINING_MODEL_PATH)

ds = Dataset.from_dict({
    "full_text": [x["full_text"] for x in data],
    "document": [str(x["document"]) for x in data],
    "tokens": [x["tokens"] for x in data],
    "trailing_whitespace": [x["trailing_whitespace"] for x in data],
    "provided_labels": [x["labels"] for x in data],
})
ds = ds.map(tokenize, fn_kwargs={"tokenizer": tokenizer, "label2id": label2id, "max_length": TRAINING_MAX_LENGTH}, num_proc=3)
# ds = ds.class_encode_column("group")

"""
--------------------------------------------
- modelden hazır tokenizer oluşturulmuş, ds oluşturulup bunun üzerinde tokenizer uygulanmış

+ "tokenizer" değişkenine daha önce eğitilmiş modeldeki tokenizer atanmış
+ sözlükten bir ds oluşturulmuş ve "ds" değişkenine atanmış
  + "full_text", "document", "tokens", "trailing_whitespace", "provided_labels" değişkenlerini içeriyor
  + her bir değişken için önceden oluşturduğumuz "data" değişkenine döngü oluşturulmuş
+ map fonksiyonu ile ds değişkenine tokenizasyon uygulanmış
  + kullanılacak işlemci çekirdeği 3 olarak belirlenmiş
--------------------------------------------
"""

In [ ]:
x = ds[0]

for t,l in zip(x["tokens"], x["provided_labels"]):
    if l != "O":
        print((t,l))

print("*"*100)

for t, l in zip(tokenizer.convert_ids_to_tokens(x["input_ids"]), x["labels"]):
    if id2label[l] != "O":
        print((t,id2label[l]))
        
"""
---------------------------------------
- ds içindeki token ve etiketleri çift halinde "o" değilse ekrana bastırıyor ardından dsdeki tokenleri input idlere dönüştürüyor, 
  karşılık gelen etiketleri yine çift halde olarak "o" değilse ekrana yazdırıyor.

+ ilk döngü veri kümesindeki öğenin tokenlerini ve sağlanan etiketlerini (provided_labels) alır. Her bir token ve etiket çifti için, 
eğer etiket "O" değilse tokeni ve etiketi yazdırır.
+ ikinci döngü veri kümesindeki öğenin tokenlerini modelin giriş kimliklerine (input_ids) dönüştürür. Daha sonra, bu giriş kimliklerine 
  karşılık gelen etiketleri (labels) alır. Her bir token ve etiket çifti için, eğer etiket "O" değilse, tokeni ve etiketi yazdırır. 
  +Burada id2label sözlüğü kullanılarak etiketlerin anlamlı hale dönüştürülmesi sağlanır.
---------------------------------------
"""

## 🧮 Competition metrics
- Note that we are not using the normal F1 score.
- Although it is early in the competition, there are plenty of discsussions already explaining this:
- e.g., here: https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data/discussion/470024

In [ ]:
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score

def compute_metrics(p, all_labels):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [all_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [all_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    
    recall = recall_score(true_labels, true_predictions)
    precision = precision_score(true_labels, true_predictions)
    f1_score = (1 + 5*5) * recall * precision / (5*5*precision + recall)
    
    results = {
        'recall': recall,
        'precision': precision,
        'f1': f1_score
    }
    return results
"""
-------------------------------------------
- modelin performansını ölçmek ve değerlendirmek için Recall, Precision ve F1 skoru hesaplanmış "results" sözlüğüne atanıp return edilmiş

+ modelden aldığımız sonuçların softmax ile en yüksek olasılıklı olanını bulup her bir etiket için o tokenin 
  en yüksek ihtimalli etiketini seçmiş ve dizi oluşturmuş bunu da predictions değişkenine atamış
+ her bir etiket için tahmin ve gerçek etiketler dolaştırılarak döngü oluşturulmuş,
  + tahmin etiketler prediction değişkenine
  + gerçek etiketler label etiketine atanır
  + bu tahmin ve etiketler "true_predictions" ve "true_labels" değişkenlerine atanmış
+ hazır kütüphaneler yardımı ile Recall, Precision ve f1 hesaplanmış ve değişkenlere atanmış
+ sonuçlar "result" adlı sözlükte birleştirilip return edilmiş
-------------------------------------------
"""

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    TRAINING_MODEL_PATH,
    num_labels=len(all_labels),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
)
collator = DataCollatorForTokenClassification(tokenizer, pad_to_multiple_of=16)

"""
---------------------------------
- önceden eğitilmiş model ve collector için "model" ve "collator" değişkenine atama yapılmış 
---------------------------------
"""

In [ ]:
# I decided to uses no eval
# final_ds = ds.train_test_split(test_size=0.2, seed=42) # cannot use stratify_by_column='group'
# final_ds

## 🏋🏻‍♀️ Training
- I actually do not use an eval set for submission to train on all data
- Values are not really tuned and go by gut feeling, as this is my first iteration / baseline

In [ ]:
# I actually chose to not use any validation set. This is only for the model I use for submission.
args = TrainingArguments(
    output_dir=OUTPUT_DIR, 
    fp16=True,
    learning_rate=2e-5,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    report_to="none",
    evaluation_strategy="no",
    do_eval=False,
    save_total_limit=1,
    logging_steps=20,
    lr_scheduler_type='cosine',
    metric_for_best_model="f1",
    greater_is_better=True,
    warmup_ratio=0.1,
    weight_decay=0.01
)

trainer = Trainer(
    model=model, 
    args=args, 
    train_dataset=ds,
    data_collator=collator, 
    tokenizer=tokenizer,
    compute_metrics=partial(compute_metrics, all_labels=all_labels),
)

"""
----------------------------------
- model için argüman ve değişken ataması yapılmış

+ modelin kaydedilceği yer OUTPUT_DIR olarak belirlenmiş
+ eğitim sırasında yarı hassas hesaplama devre dışı bırakılmış
  + fp16 hesaplama yapılırken daha az bellek kullanımı ve daha hızlı işlem yapılmasını sağlayan bir tekniktir
+ Eğitimde verilerin 3 tur eğitilceği belirtilmiş, Her bir tur, tüm eğitim verilerinin bir kez model tarafından geçirilmesini ifade eder.
+ eğitim sırasındaki raporların gönderilmesi istenmemiş
+ eğitim sırasında değerlendirme yapılmaması adına ayarlar yapılmış
+ en iyi modelin belirlenmesinde kullanılcak olan metrik f1 olarak belirlenmiş
+ trainer sınıfındaki değişkenler için yollar tek tek atanmış
----------------------------------
"""

In [ ]:
%%time
trainer.train()
"""
---------------------------------
- eğitim süresini ölçmek için time komutu kullanılarak eğitim başlatılmış.
---------------------------------
"""

## 💾 Save models
- You can click on "Save version" (top right) and "Save & Run All (Commit)"
- Then you can use this notebook as input for your inference notebook

In [ ]:
trainer.save_model("deberta3base_1024")
tokenizer.save_pretrained("deberta3base_1024")

"""
-----------------------------------------
- trainer modeli ve tokenizer "deberta3base_1024" dizinine kaydedilmiş
-----------------------------------------
"""